In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 로드
# 데이터 로드
data = pd.read_csv('AllLeftArm.csv')
df = pd.DataFrame(data)

# 특성과 타겟 변수 설정
X = df[['no','sand']]
y = df['state']

# 레이블 인코딩
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Grid Search를 위한 파라미터 그리드 정의
param_grid = {
    'criterion': ['gini', 'entropy'],  # 분할 기준
    'max_depth': [None, 10, 20, 30, 40, 50],  # 최대 깊이
    'min_samples_split': [2, 5, 10],  # 분할을 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4]  # 리프 노드의 최소 샘플 수
}

# Grid Search 초기화
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, n_jobs=-1)

# Grid Search를 사용하여 전체 데이터에 적합
grid_search.fit(X_scaled, y_encoded)

# Grid Search에서 가장 좋은 모델 가져오기
best_model = grid_search.best_estimator_

# 최적의 파라미터와 점수 출력
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
# 교차 검증 수행 (여기서는 5-Fold 교차 검증을 사용)
cv_scores = cross_val_score(best_model, X_scaled, y_encoded, cv=5)

print(f"Cross-Validation Scores: {cv_scores}")
print(f"Mean Cross-Validation Score: {cv_scores.mean()}")
# 전체 데이터에 대해 예측
y_pred = best_model.predict(X_scaled)

# 예측된 레이블을 원래 레이블로 변환
y_pred_labels = le.inverse_transform(y_pred)
y_true_labels = le.inverse_transform(y_encoded)

# 모델 평가
conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_true_labels, y_pred_labels))


Best Parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2}
Best Score: 0.599047619047619
Cross-Validation Scores: [0.53333333 0.53333333 0.57142857 0.64285714 0.71428571]
Mean Cross-Validation Score: 0.599047619047619
Confusion Matrix:
[[30  6]
 [ 8 28]]

Classification Report:
              precision    recall  f1-score   support

          no       0.79      0.83      0.81        36
        sand       0.82      0.78      0.80        36

    accuracy                           0.81        72
   macro avg       0.81      0.81      0.81        72
weighted avg       0.81      0.81      0.81        72

